<a href="https://colab.research.google.com/github/zcongfly/huggingface-nlp-learning-note/blob/main/00_transformer%E6%A8%A1%E5%9E%8B%E4%BB%8B%E7%BB%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# transformer模型介绍

声明：因为我之前已经跟着李沐大神的课程学过一遍了，这里的transformer模型不会介绍得特别详细，只大致记录我不太熟悉的部分内容。详情请看我的另一个笔记：

[]

## 简介

transformer模型主要由两个块组成：

* Encoder (左侧): 编码器接收输入并构建其表示（其特征）。这意味着对模型进行了优化，以从输入中获得理解。
* Decoder (右侧): 解码器使用编码器的表示（特征）以及其他输入来生成目标序列。这意味着该模型已针对生成输出进行了优化。

这些部件中的每一个都可以独立使用，具体取决于任务：

* Encoder-only models: 适用于需要理解输入的任务，如句子分类和命名实体识别。
* Decoder-only models: 适用于生成任务，如文本生成。
* Encoder-decoder models 或者 sequence-to-sequence models: 适用于需要根据输入进行生成的任务，如翻译或摘要。

在后面的部分中，我们将单独地深入研究这些体系结构。


## 原始的结构

Transformer架构最初是为翻译而设计的。在训练期间，编码器接收特定语言的输入（句子），而解码器需要输出对应语言的翻译。在编码器中，注意力层可以使用一个句子中的所有单词（正如我们刚才看到的，给定单词的翻译可以取决于它在句子中的其他单词）。然而，解码器是按顺序工作的，并且只能注意它已经翻译过的句子中的单词。例如，当我们预测了翻译目标的前三个单词时，我们将它们提供给解码器，然后解码器使用编码器的所有输入来尝试预测第四个单词。

为了在训练过程中加快速度（当模型可以访问目标句子时），解码器会被输入整个目标，但不允许获取到要翻译的单词（如果它在尝试预测位置2的单词时可以访问位置2的单词，解码器就会偷懒，直接输出那个单词，从而无法学习到正确的语言关系！）。例如，当试图预测第4个单词时，注意力层只能获取位置1到3的单词。

最初的Transformer架构如下所示，编码器位于左侧，解码器位于右侧：

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter1/transformers.svg)

注意，解码器块中的第一个注意力层关联到解码器的所有（过去的）输入，但是第二注意力层使用编码器的输出。因此，它可以访问整个输入句子，以最好地预测当前单词。这是非常有用的，因为不同的语言可以有语法规则将单词按不同的顺序排列，或者句子后面提供的一些上下文可能有助于确定给定单词的最佳翻译。

也可以在编码器/解码器中使用注意力遮罩层，以防止模型注意某些特殊单词。例如，在批处理句子时，填充特殊词使所有句子的长度一致。




## 架构与参数

在本课程中，当我们深入探讨Transformers模型时，您将看到 架构、参数和模型 。 这些术语的含义略有不同：

* 架构: 这是模型的骨架 — 每个层的定义以及模型中发生的每个操作。
* Checkpoints: 这些是将在给架构中结构中加载的权重。
* 模型: 这是一个笼统的术语，没有“架构”或“参数”那么精确：它可以指两者。为了避免歧义，本课程使用将使用架构和参数。

例如，BERT是一个架构，而 bert-base-cased， 这是谷歌团队为BERT的第一个版本训练的一组权重参数，是一个参数。我们可以说“BERT模型”和”bert-base-cased模型.”

## 编码器模型

“编码器”模型指仅使用编码器的Transformer模型。在每个阶段，注意力层都可以获取初始句子中的所有单词。这些模型通常具有“双向”注意力，被称为自编码模型。

这些模型的预训练通常围绕着以某种方式破坏给定的句子（例如：通过随机遮盖其中的单词），并让模型寻找或重建给定的句子。

“编码器”模型最适合于需要理解完整句子的任务，例如：句子分类、命名实体识别（以及更普遍的单词分类）和阅读理解后回答问题。


## 解码器模型

“解码器”模型通常指仅使用解码器的Transformer模型。在每个阶段，对于给定的单词，注意力层只能获取到句子中位于将要预测单词前面的单词。这些模型通常被称为自回归模型。

“解码器”模型的预训练通常围绕预测句子中的下一个单词进行。

这些模型最适合于涉及文本生成的任务。


## 序列到序列模型

编码器-解码器模型（也称为序列到序列模型)同时使用Transformer架构的编码器和解码器两个部分。在每个阶段，编码器的注意力层可以访问初始句子中的所有单词，而解码器的注意力层只能访问位于输入中将要预测单词前面的单词。

这些模型的预训练可以使用训练编码器或解码器模型的方式来完成，但通常涉及更复杂的内容。例如，T5通过将文本的随机跨度（可以包含多个单词）替换为单个特殊单词来进行预训练，然后目标是预测该掩码单词替换的文本。

序列到序列模型最适合于围绕根据给定输入生成新句子的任务，如摘要、翻译或生成性问答。


## 偏见和局限性

如果您打算在正式的项目中使用经过预训练或经过微调的模型。请注意：虽然这些模型是很强大，但它们也有局限性。其中最大的一个问题是，为了对大量数据进行预训练，研究人员通常会搜集所有他们能找到的内容，中间可能夹带一些意识形态或者价值观的刻板印象。

为了快速解释清楚这个问题，让我们回到一个使用BERT模型的pipeline的例子：

In [ ]:
!pip install transformers

from transformers import pipeline

unmasker=pipeline("fill-mask",model="bert-base-uncased")
result=unmasker("This man works as a [MASK].")
print([r["token_str"] for r in result])

result = unmasker("This woman works as a [MASK].")
print([r["token_str"] for r in result])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.4 MB/s eta 0:00:00


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['carpenter', 'lawyer', 'farmer', 'businessman', 'doctor']
['nurse', 'maid', 'teacher', 'waitress', 'prostitute']


当要求模型填写这两句话中缺少的单词时，模型给出的答案中，只有一个与性别无关（waiter/waitress）。其他职业通常与某一特定性别相关，prostitute最终进入了模型中与“woman”和“work”相关的前五位。尽管BERT是使用经过筛选和清洗后，明显中立的数据集上建立的的Transformer模型，而不是通过从互联网上搜集数据（它是在Wikipedia 英文和BookCorpus数据集）。

因此，当您使用这些工具时，您需要记住，使用的原始模型的时候，很容易生成性别歧视、种族主义或恐同内容。这种固有偏见不会随着微调模型而使消失。

## 总结

| 模型<br/>          | 示例                                       | 任务                                     |
| ------------------ | ------------------------------------------ | ---------------------------------------- |
| 编码器<br/>        | ALBERT, BERT, DistilBERT, ELECTRA, RoBERTa | 句子分类、命名实体识别、从文本中提取答案 |
| 解码器<br/>        | CTRL, GPT, GPT-2, Transformer XL           | 文本生成                                 |
| 编码器-解码器<br/> | BART, T5, Marian, mBART                    | 文本摘要、翻译、生成问题的回答           |

## 章节测试

https://huggingface.co/learn/nlp-course/zh-CN/chapter1/10?fw=pt

**1.“迁移学习”是什么意思？**

A. 通过在同一数据集上训练模型，将预训练模型的知识迁移到新模型。

B. 通过使用第一个模型的权重初始化第二个模型，将预训练模型的知识迁移到新模型。

C. 构建与第一个模型具有相同架构的第二个模型，将预训练模型的知识迁移到新模型。

答案：B

解析：

A. 错误，那将是同一模型的两个版本。

B. 正确：当第二个模型接受新任务训练时，它*迁移*第一个模型的知识。

C. 错误，架构只是模型的构建方式； 在这种情况下，没有知识共享或迁移。

**2.语言模型在预训练时通常不需要标签，这样的说法是否正确。**

正确，预训练通常是自监督，这意味着标签是根据输入自动创建的（例如：预测下一个单词或填充一些[MARSK]单词）。

**3.选择最能描述“模型(model)”、“架构(architecture)”和“权重(weights)”的句子。**

A. 如果模型是一座建筑物，那么它的架构就是蓝图，而权重就是住在里面的人。

B. 架构是构建模型的地图，其权重是地图上表示的城市。

C. 架构是用于构建模型的一系列数学函数，其权重是这些函数参数。

答案：C

解析：同一组数学函数（架构）可以通过使用不同的参数（权重）来构建不同的模型。